In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# PostgreSQL connection details
pg_user = "root"
pg_password = "root"
pg_host = "localhost"
pg_port = "5432"
pg_db = "ny_taxi"

# Create SQLAlchemy engine
url = f"postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}"
engine = create_engine(url)

In [11]:
# For the trips in November 2025, how many trips had a trip_distance of less than or equal to 1 mile?

pd.read_sql(
    """
    SELECT COUNT(1)
    FROM green_taxi_data_2025
    WHERE trip_distance <= 1
    """,
    engine
)

,count
0,8009


In [24]:
# Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles?

pd.read_sql(
    """
    WITH subquery AS (
        SELECT *
        FROM green_taxi_data_2025
        WHERE trip_distance < 100
    )
    SELECT lpep_pickup_datetime
    FROM subquery
    WHERE trip_distance = (SELECT MAX(trip_distance) FROM subquery)
    """,
    engine
)

,lpep_pickup_datetime
0,2025-11-14 15:36:27


In [47]:
# Which was the pickup zone with the largest total_amount (sum of all trips) on November 18th, 2025?

pd.read_sql(
    """
    WITH subquery AS (
        SELECT *
        FROM green_taxi_data_2025
        WHERE lpep_pickup_datetime::DATE = '2025-11-18'
    )
    SELECT "PULocationID", "Zone", SUM(total_amount) AS total_amount
    FROM subquery a
    JOIN taxi_zone_lookup b
    ON a."PULocationID" = b."LocationID"
    GROUP BY "PULocationID", "Zone"
    ORDER BY total_amount DESC
    LIMIT 1
    """,
    engine
)

,PULocationID,Zone,total_amount
0,74,East Harlem North,9281.92


In [60]:
# For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

pd.read_sql(
    """
    WITH subquery AS (
        SELECT "PULocationID", "DOLocationID", tip_amount
        FROM green_taxi_data_2025
        WHERE lpep_pickup_datetime::DATE = '2025-11-25'
        AND "PULocationID" = 74
    )
    SELECT "DOLocationID", "Zone", SUM(tip_amount) AS tip_amount
    FROM subquery a
    JOIN taxi_zone_lookup b
    ON a."DOLocationID" = b."LocationID"
    GROUP BY "DOLocationID", "Zone"
    ORDER BY tip_amount DESC
    LIMIT 1
    """,
    engine
)

,DOLocationID,Zone,tip_amount
0,75,East Harlem South,148.1
